In [ ]:
import numpy as np

from datasets import load_dataset
from huggingface_hub import login

#
import warnings
warnings.filterwarnings('ignore')

import sys
import os
dir2 = os.path.abspath('')
dir1 = os.path.dirname(dir2)
if not dir1 in sys.path: sys.path.append(dir1)
from access_tokens import HF_WRITE_TOKEN

In [ ]:
login(token=HF_WRITE_TOKEN)

In [ ]:
envs = {
    0: "walker2d-expert-v2",
    1: "halfcheetah-expert-v2",
}

chosen_env = envs[1]

In [ ]:
dataset = load_dataset("edbeeching/decision_transformer_gym_replay", chosen_env)['train']

In [ ]:
total_max = 0
total_min = 0

for t in dataset:
    total_max = max(total_max, np.sum(t['rewards']))
    total_min = min(total_min, np.sum(t['rewards']))
    
print("Max total reward:", total_max)
print("Min total reward:", total_min)

In [ ]:
# run if we want to add a 0 adversary
pr_actions = dataset['actions']
new_pr_actions = []
adv_actions =[]
for tr in pr_actions:
    pr_l = []
    adv_l = []
    for a in tr:
        adv = np.array(a) * 0
        pr = np.array(a)
        pr_l.append(list(pr))
        adv_l.append(list(adv))
    new_pr_actions.append(pr_l)
    adv_actions.append(adv_l)

dataset = dataset.add_column('pr_actions', new_pr_actions)
dataset = dataset.add_column('adv_actions', adv_actions)
dataset = dataset.remove_columns(['actions'])
dataset.save_to_disk(f'./datasets/d4rl_{chosen_env.replace("-v2", "").replace("-", "_")}')

In [ ]:
# # run if we want to add a noisy random adversary
# pr_actions = dataset['actions']
# new_pr_actions = []
# adv_actions =[]
# for tr in pr_actions:
#     pr_l = []
#     adv_l = []
#     for a in tr:
#         adv = np.array(a) * 0.1 * np.random.choice([-1, 1], size=len(a)) * np.random.rand(len(a))
#         pr = np.array(a) + adv
#         pr_l.append(list(pr))
#         adv_l.append(list(adv))
#     new_pr_actions.append(pr_l)
#     adv_actions.append(adv_l)

# dataset = dataset.add_column('pr_actions', new_pr_actions)
# dataset = dataset.add_column('adv_actions', adv_actions)
# dataset = dataset.remove_columns(['actions'])
# dataset.save_to_disk(f'./datasets/d4rl_{chosen_env.replace("-v2", "").replace("-", "_")}_noisy')